In [1]:
import os
import json
from datetime import datetime

from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, Markdown
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path="../../.env")

# Initialize OpenAI client using key from .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Load model name from .env, defaulting to gpt-4o
MODEL_NAME = os.getenv("OPENAI_MODEL", "gpt-4o")

# Editable system prompt
system_prompt_input = widgets.Textarea(
    value="You are ChatGPT, a helpful assistant.",
    description="System Prompt:",
    layout=widgets.Layout(width="100%", height="60px"),
    style={"description_width": "initial"}
)

# Chat session state
class ChatSession:
    def __init__(self, session_id=None, system_prompt=None):
        self.session_id = session_id or datetime.now().strftime("%Y%m%d_%H%M%S")
        self.system_prompt = system_prompt or "You are ChatGPT, a helpful assistant."
        self.history = [{"role": "system", "content": self.system_prompt}]

    def add_user_message(self, content):
        self.history.append({"role": "user", "content": content})

    def add_assistant_message(self, content):
        self.history.append({"role": "assistant", "content": content})

    def save_to_file(self):
        os.makedirs("../chat_sessions", exist_ok=True)
        with open(f"../chat_sessions/{self.session_id}.json", "w") as f:
            json.dump(self.history, f, indent=2)

# Query OpenAI with history
def query_chatgpt(messages):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )
    return response.choices[0].message.content

# Optional: AI-suggested next prompt
def suggest_next_prompt(history):
    prompt = [
        {"role": "system", "content": "You are a helpful assistant that suggests what the user might want to ask next, based on the recent conversation."},
        *history[-6:],  # recent context only
        {"role": "user", "content": "Suggest the next best user prompt."}
    ]
    try:
        suggestion = query_chatgpt(prompt)
        return suggestion
    except Exception as e:
        return f"(Could not generate suggestion: {e})"

# Widgets
output_box = widgets.Output()
input_box = widgets.Textarea(
    placeholder="Type your message here...",
    layout=widgets.Layout(width="100%", height="80px")
)
suggestion_box = widgets.HTML(value="", placeholder="(Next best prompt)", layout=widgets.Layout(margin="8px 0 0 0"))
send_btn = widgets.Button(description="Send", button_style="primary")
new_chat_btn = widgets.Button(description="New Chat", button_style="warning")
session_label = widgets.Label()

# Init session
current_session = ChatSession(system_prompt=system_prompt_input.value)
session_label.value = f"Current Session: {current_session.session_id}"

# Send handler
def on_send(_):
    user_input = input_box.value.strip()
    if not user_input:
        return
    input_box.value = ""
    current_session.add_user_message(user_input)

    with output_box:
        display(Markdown(f"**You:** {user_input}"))

    try:
        response = query_chatgpt(current_session.history)
    except Exception as e:
        response = f"Error: {str(e)}"

    current_session.add_assistant_message(response)
    current_session.save_to_file()

    with output_box:
        display(Markdown(f"**OpenAI:** {response}"))

    suggestion = suggest_next_prompt(current_session.history)
    suggestion_box.value = f"<b>💡 Suggested Next Prompt:</b> <i>{suggestion}</i>"

# New chat handler
def on_new_chat(_):
    global current_session
    current_session = ChatSession(system_prompt=system_prompt_input.value)
    session_label.value = f"Current Session: {current_session.session_id}"
    output_box.clear_output()
    input_box.value = ""
    suggestion_box.value = ""

# Bind buttons
send_btn.on_click(on_send)
new_chat_btn.on_click(on_new_chat)

# Render UI
ui = widgets.VBox([
    system_prompt_input,
    session_label,
    output_box,
    widgets.HBox([send_btn, new_chat_btn]),
    input_box,
    suggestion_box
])

display(ui)
